In [1]:
%matplotlib inline

In [2]:
# Imports
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (30, 15)
seed = 42
np.random.seed(seed)

In [3]:
# Courtesy of https://www.kaggle.com/mhdzahier/travel-insurance
df = pd.read_csv('data/travel insurance.csv')

In [25]:
# Examine data

In [26]:
# %load snippets/clean.py

In [27]:
# Train Logistic Regression

In [28]:
# %load snippets/train_test.py

In [9]:
# Metric of choice

In [29]:
# %load snippets/class_balance.py

In [30]:
# %load snippets/roc_auc_curve.py

In [31]:
# %load snippets/other_metrics.py

In [32]:
# %load snippets/all_the_models.py

In [33]:
# %load snippets/different_parameters.py

In [34]:
# %load snippets/gridsearch.py

In [35]:
# %load snippets/cv_demonstration.py

In [36]:
# %load snippets/cv_hard_way.py

In [37]:
# %load snippets/cv_easy_way.py

In [38]:
# %load snippets/feature_importance_gb

In [39]:
# %load snippets/feature_importance_lr

# Conclusion
We have covered the following topics:
- Choosing the right metric for the job
- Train/test splitting
- Hyperparameters
- Grid searching
- Cross validation
- Feature Importances

There are still many aspects to Data Science and scikit-learn that I have not covered here, such as data pipelines, preprocessing and other cool features of sklearn. Hopefully it's enough to get you started, and help you not make simple, but devious mistakes!

It's never been easier to train a model, but that just makes it more dangerous if you're not careful!